In [1]:
import numpy as np
import networkx as nx

import mmk

In [2]:
chain = mmk.MarkovChain(
    {
        "start": {
            "try": 1.
        },
        "try": {
            "lost": .1,
            "delivered": .9,
        },
        "lost": {
            "try": 1.,
        },
        "delivered": {
            "start": 1.,
        },
    }
)

In [3]:
formula = "eventually delivered"  # or "<> delivered" 
p = mmk.probability_of_formula(chain, formula)

for s, p in zip(chain._states, p):
    print(f"ℙ ( {s} ⊨ {formula} ) = {p}")

ℙ ( delivered ⊨ eventually delivered ) = 1.0
ℙ ( lost ⊨ eventually delivered ) = 1.0
ℙ ( start ⊨ eventually delivered ) = 1.0
ℙ ( try ⊨ eventually delivered ) = 1.0
